This project aims at simulating on your computer the paramagnetic/ferromagnetic phase transition. For this, we will simulate the 2 dimensional ising model.

# Ising model

The Hamiltonian (without external magnetic field) is

$$H = -J\sum_{(i,j)}S_i S_j$$

where $S_i$ is the spin at location $i$ and $J$ is a constant describing the strength of the interaction between neighbouring spins. We consider $S_i = \pm 1$ (for all $i$). The sum in the previous equation runs overs all neaghbouring spins. In order to simulate the system, we will implement the Metropolis algorithm, which proceeds as follows:

1. Start with some random spin configuration
2. Pick a spin at random and flip it
3. Compute the energy change $\Delta E$ due to this flip
4. If $\Delta E \leq 0$, accept the flip
5. If $\Delta E > 0$, accept the flip with probability $P = e^{-\beta \Delta E}$, where $\beta = \frac{1}{k_BT}$
6. If flip is rejected, put the spin back in its original direction
7. Go to 2. until maximum number of iteractions is reached

# Implementation

In order to simplify the implementation, we will take $J=1$ and $k_B=1$. 

## Step 1

The first step of the implementation consists in creating a 2D grid of spins in up and down positions and plotting it. For this, do the following:

- Implement a `struct` called `Grid` having as fields: a 2D slice (that will contain the spin values at each node of the grid), the size $N$ of the grid ($N$ corresponds to the number of rows and columns, so that the total number of spins is $N\times N$), the coupling $J$ and the temperature $T$.
- Implement a `NewGrid` function that builds an object of type `Grid` (take $N=80$ and $T=4$). 
- Implement a method called `Init` that randomly initializes the grid.

In the last step, one need to draw random numbers. For this, the `math/rand` package can be used. Once this is done, a plot displaying the grid must be produced. For this, we will use the `gonum/plot` package. The idea is to create two scatter plots (one containing the coordinates of the up spins and the other one containing the coordinates of the down spins) and to plot them on top of each other. Scatter plots can be created using the `plotter.Scatter` structure. This structure requires a `struct` implementing the `XYer` interface (defined in plotter/plotter.go). You thus need to implement such a `struct` and create two instances of it, one for the up spins and one for the down spins. 

Now that you have your two `plotter.Scatter` objects `scaUp` and `scaDown` you can plot them. For this, do:

```
p, _ := plot.New()
p.Add(scaUp, scaDown)
p.Save(6*vg.Inch, 6*vg.Inch, "Grid2D.png")
```

The first line creates a plot object. The second line adds to the plot your two scatter plots and the third line saves your plot into a png file. Open this png file and check that it makes sense. 

## Step 3

Add an `SpinEnergy` method to the `Grid` struct that computes the energy of a spin. In order to compute this energy, we consider that the spin only interacts with its four nearest neighbours (top, right, bottom and left neighbours) and that the grid has a periodic structure (it is folded on a cylinder).

## Dynamic plotting

Implement remaing steps. Once this is done, repeat steps 2->6 many times (about $7\times10^{6}$ times) using a `for` loop. In order to observe the effect of this repetition, we will improve the plotting of the grid. The grid plot you performed above was sufficient to check that the grid was properly implemented but doesn't allow to observe the evolution of the grid as the `for` loop progresses. For this, a more refined way of plotting using a web server will be used. As this is a bit subtle, we provide you with the functions that allow you to do it (implemented in the plot.go file). We don't expect you to understand this code now. In order to use it do the following:

- launch a web server adding, at the beginning of your `main` function the following line

    ```
    go webServer(addrFlag)
    ```

   the addrFlag is a `string` containing the url where the plot content will be served (e.g. ":5555/"). In order to optimize your code, use the flag package to set this string to a proper value. 

- perform the plot invoking

    ```
    datac <- Plots{Plot: renderSVG(p)}
    ```

   where p is the same plot object as the one used previously. In short, the command encodes the plots as svg and sends this svg to the web, where it is displayed. In order to optimize the plotting, don't plot the grid every iteration. You can for example plot it every $40 000$ iteration.

- when the code is running, open a web browser at the address defined above (e.g. http://localhost:5555/ if the addrFlag was ":5555/").

# Studies 


## Qualitative studies

- Observe the evolution of the grid at $T=4$. Does your observation makes sense ?
- Decrease the temperature progressively down to 1 (by steps of 1 for example). What do you observe ?
- Change the sign of $J$ and run a few simulations. What do you observe ?
- Add the effect of an external magnetic field $h$ to your simulation (for this, you should account for a new term in the Hamiltonian which now is $H = -J\sum_{(i,j)}S_i S_j - h \sum_{i} S_i$) and run your simulation several times by varying $h$. What do you observe ?

## Quantitative studies

In what follows, the external magnetic field should be set to $0$. We will also reduce the size of the grid to $N=16$ in order to reduce computing time. 

- Plot the average energy and the average magnetisation of the grid as a function of the temperature between $1$ and $4$ (a total of $200$ points or more between these two values would be perfect !). You should obtain something like this:

<img src="energyVstemp.png" height="200" width="300"  />
<img src="magVstemp.png" height="200" width="300" />


- Calculate the specific heat $C_V$ using the following formula (for a proof, see the book "Physique Statistique" by B. Diu et al. at pages 269 and 270):

    $$C_V = \frac{1}{k_B T^2}\left(<E^2> - <E>^2\right)$$

   where $<E>$ denotes the average energy (the one you calculated in the previous question).
   
   Plot $C_V$ as a function of the temperature. You should obtain something like this:
   
   <img src="CvVstemp.png" height="200" width="300"  />
   
What do you conclude ?